In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bayesian import BayesianLinearRegression
from sklearn.model_selection import train_test_split

sns.set_theme()
sns.set_context("notebook")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
dtype_dict = {
    'Date': 'str',
    'FarmName_Pseudo': 'str',
    'SE_Number': 'str',
    'Age': 'Int64',
    'BreedName': 'str',
    'DailyYield': 'float',
    'PreviousDailyYield': 'float',
    'DailyYieldChange': 'float',
    'DaysInMilk': 'Int64',
    'YearSeason': 'str',
    'LactationNumber': 'Int64',
    'ExpectedYield': 'float',
    'NormalizedDailyYield': 'float',
    'NormalizedDailyYieldChange': 'float',
    'HeatStress': 'Int64',
    'Temp15Threshold': 'Int64',
    'HW': 'Int64',
    'cum_HW': 'Int64',
    'MeanTemperature': 'float',
    'MeanTHI_adj': 'float',
    'HeatLoad': 'float',
    'CumulativeHeatLoad': 'float',
}

milk_data = pd.read_csv('../Data/MergedData/HeatApproachYieldDataTest.csv', dtype=dtype_dict)
milk_data['Date'] = pd.to_datetime(milk_data['Date'], format='%Y-%m-%d')
milk_data.head(-5)

,Date,FarmName_Pseudo,SE_Number,Age,BreedName,LactationNumber,DaysInMilk,YearSeason,DailyYield,PreviousDailyYield,...,NormalizedDailyYieldChange,Residuals,HeatStress,Temp15Threshold,HW,cum_HW,MeanTemperature,MeanTHI_adj,HeatLoad,CumulativeHeatLoad
0,2022-05-28,a624fb9a,SE-064c0cec-1189,3242,02 SLB,8,3,2022-2,15.22,0.0000,...,0.000000,1.820438,0,0,0,0,9.912500,50.478673,-12.521327,0.0
1,2022-05-29,a624fb9a,SE-064c0cec-1189,3243,02 SLB,8,4,2022-2,18.96,15.2200,...,0.107655,1.589745,0,0,0,0,10.066667,53.841648,-9.158352,0.0
2,2022-05-30,a624fb9a,SE-064c0cec-1189,3244,02 SLB,8,5,2022-2,22.64,17.0900,...,0.089176,1.894598,0,1,0,0,10.466667,52.935959,-10.064041,0.0
3,2022-05-31,a624fb9a,SE-064c0cec-1189,3245,02 SLB,8,6,2022-2,26.49,18.9400,...,0.079936,2.877443,0,0,0,0,11.183333,52.872112,-10.127888,0.0
4,2022-06-01,a624fb9a,SE-064c0cec-1189,3246,02 SLB,8,7,2022-3,33.61,20.8275,...,0.098152,7.563598,0,1,0,0,12.704167,56.056547,-6.943453,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465310,2023-06-03,f454e660,SE-fcdf259d-0044-0,4150,41 Fjällko,10,347,2023-3,12.67,14.6520,...,-0.048011,-0.285470,0,1,0,0,12.666667,53.132530,-9.867470,0.0
465311,2023-06-04,f454e660,SE-fcdf259d-0044-0,4151,41 Fjällko,10,348,2023-3,22.31,14.0300,...,0.074182,9.449823,0,1,0,0,13.079167,56.726870,-6.273130,0.0
465312,2023-06-05,f454e660,SE-fcdf259d-0044-0,4152,41 Fjällko,10,349,2023-3,12.84,14.9840,...,-0.007207,0.075116,0,1,0,0,14.237500,58.482418,-4.517582,0.0
465313,2023-06-06,f454e660,SE-fcdf259d-0044-0,4153,41 Fjällko,10,350,2023-3,9.47,14.8920,...,-0.022416,-3.199590,0,1,0,0,15.345833,60.546358,-2.453642,0.0


In [15]:
from sklearn.preprocessing import StandardScaler

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['FarmName_Pseudo', 'FarmHeatLoadMilkProduction'])

# Set subject type to 'farm'
subject_type = 'farm'

# Specify the farm ID for analysis
farm_id = 'a624fb9a'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Scale the HeatLoad feature
scaler = StandardScaler()
train_data['HeatLoad'] = scaler.fit_transform(train_data[['HeatLoad']])
val_data['HeatLoad'] = scaler.transform(val_data[['HeatLoad']])

# Define the single feature
features = ['HeatLoad']

# Calculate the correlation between HeatLoad and NormalizedDailyYield
heatload_correlation = train_data['HeatLoad'].corr(train_data[target])
print("Correlation between HeatLoad and NormalizedDailyYield:", heatload_correlation)

# Function to fit and get the posterior mean for HeatLoad
def fit_and_get_heatload_posterior(farm_id, train_data, features, heatload_correlation):
    print(f"\nSelected features: {features}")

    # Calculate prior means for the features
    prior_mean_values = [heatload_correlation]  # Use correlation as prior mean for HeatLoad
    prior_std_values = [train_data[feature].std() for feature in features]
    
    # Define priors
    prior_mean = np.array([normalized_mean] + prior_mean_values).reshape(-1, 1)
    prior_std = np.array([np.sqrt(normalized_variance)] + prior_std_values).reshape(-1, 1)
    prior_cov = np.eye(len(features) + 1) * 0.01  # Smaller value for stronger prior confidence
    beta = 1 / normalized_variance

    # Print prior means and standard deviations
    print(f"\nPrior Mean and Standard Deviation for each feature:\n")
    for feature, mean, std in zip(['Off-set'] + features, prior_mean.flatten(), prior_std.flatten()):
        print(f"{feature}: mean = {mean}, std_dev = {std}")

    # Initialize and fit the model
    model = BayesianLinearRegression(
        dataframe=train_data,
        subject_name=farm_id,
        selected_features=features,
        target=target,
        subject_type=subject_type,
        prior_mean=prior_mean,
        prior_cov=prior_cov,
        beta=beta
    )
    result = model.fit_model()

    # Print posterior mean and standard deviation for each feature
    print(f"\nPosterior Mean and Standard Deviation for each feature:\n")
    for feature, stats in result.items():
        print(f"{feature}: mean = {stats['mu']}, std_dev = {stats['sigma']}")

    # Extract the posterior mean for HeatLoad
    heatload_posterior_mean = result['HeatLoad']['mu']

    return heatload_posterior_mean

# Train the model with the HeatLoad feature on the training set and print results
heatload_posterior_mean = fit_and_get_heatload_posterior(farm_id, train_data, features, heatload_correlation)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmHeatLoadMilkProduction': heatload_posterior_mean
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 1.0006942911285428
Standard Deviation of NormalizedDailyYield: 0.17117145952708357
Variance of NormalizedDailyYield: 0.029299668556632005
Correlation between HeatLoad and NormalizedDailyYield: -0.037973918829283866

Selected features: ['HeatLoad']

Prior Mean and Standard Deviation for each feature:

Off-set: mean = 1.0006942911285428, std_dev = 0.17117145952708357
HeatLoad: mean = -0.037973918829283866, std_dev = 1.0000123328133002

Posterior Mean and Standard Deviation for each feature:

Off-set: mean = 1.0005480608213502, std_dev = 0.0008503306871013713
HeatLoad: mean = -0.006501740633351767, std_dev = 0.0008503306871013692


,FarmName_Pseudo,FarmHeatLoadMilkProduction
0,a624fb9a,-0.006502


In [16]:
from sklearn.preprocessing import StandardScaler

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['FarmName_Pseudo', 'FarmCumulativeHeatLoadMilkProduction'])

# Set subject type to 'farm'
subject_type = 'farm'

# Specify the farm ID for analysis
farm_id = 'a624fb9a'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Scale the CumulativeHeatLoad feature
scaler = StandardScaler()
train_data['CumulativeHeatLoad'] = scaler.fit_transform(train_data[['CumulativeHeatLoad']])
val_data['CumulativeHeatLoad'] = scaler.transform(val_data[['CumulativeHeatLoad']])

# Define the single feature
features = ['CumulativeHeatLoad']

# Calculate the correlation between CumulativeHeatLoad and NormalizedDailyYield
cumulative_heatload_correlation = train_data['CumulativeHeatLoad'].corr(train_data[target])
print("Correlation between CumulativeHeatLoad and NormalizedDailyYield:", cumulative_heatload_correlation)

# Function to fit and get the posterior mean for CumulativeHeatLoad
def fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation):
    print(f"\nSelected features: {features}")

    # Calculate prior means for the features
    prior_mean_values = [cumulative_heatload_correlation]  # Use correlation as prior mean for CumulativeHeatLoad
    prior_std_values = [train_data[feature].std() for feature in features]
    
    # Define priors
    prior_mean = np.array([normalized_mean] + prior_mean_values).reshape(-1, 1)
    prior_std = np.array([np.sqrt(normalized_variance)] + prior_std_values).reshape(-1, 1)
    prior_cov = np.eye(len(features) + 1) * 0.01  # Smaller value for stronger prior confidence
    beta = 1 / normalized_variance

    # Print prior means and standard deviations
    print(f"\nPrior Mean and Standard Deviation for each feature:\n")
    for feature, mean, std in zip(['Off-set'] + features, prior_mean.flatten(), prior_std.flatten()):
        print(f"{feature}: mean = {mean}, std_dev = {std}")

    # Initialize and fit the model
    model = BayesianLinearRegression(
        dataframe=train_data,
        subject_name=farm_id,
        selected_features=features,
        target=target,
        subject_type=subject_type,
        prior_mean=prior_mean,
        prior_cov=prior_cov,
        beta=beta
    )
    result = model.fit_model()

    # Print posterior mean and standard deviation for each feature
    print(f"\nPosterior Mean and Standard Deviation for each feature:\n")
    for feature, stats in result.items():
        print(f"{feature}: mean = {stats['mu']}, std_dev = {stats['sigma']}")

    # Extract the posterior mean for CumulativeHeatLoad
    cumulative_heatload_posterior_mean = result['CumulativeHeatLoad']['mu']

    return cumulative_heatload_posterior_mean

# Train the model with the CumulativeHeatLoad feature on the training set and print results
cumulative_heatload_posterior_mean = fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmCumulativeHeatLoadMilkProduction': cumulative_heatload_posterior_mean
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 1.0006942911285428
Standard Deviation of NormalizedDailyYield: 0.17117145952708357
Variance of NormalizedDailyYield: 0.029299668556632005
Correlation between CumulativeHeatLoad and NormalizedDailyYield: -0.06980560554887

Selected features: ['CumulativeHeatLoad']

Prior Mean and Standard Deviation for each feature:

Off-set: mean = 1.0006942911285428, std_dev = 0.17117145952708357
CumulativeHeatLoad: mean = -0.06980560554887, std_dev = 1.0000123328133002

Posterior Mean and Standard Deviation for each feature:

Off-set: mean = 1.0005480608213502, std_dev = 0.0008503306871013713
CumulativeHeatLoad: mean = -0.011951833153517099, std_dev = 0.0008503306871013688


,FarmName_Pseudo,FarmCumulativeHeatLoadMilkProduction
0,a624fb9a,-0.011952


In [17]:
from sklearn.preprocessing import StandardScaler

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['FarmName_Pseudo', 'FarmCumulativeHeatLoadMilkProduction'])

# Set subject type to 'farm'
subject_type = 'farm'

# Specify the farm ID for analysis
farm_id = '5c06d92d'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Scale the CumulativeHeatLoad feature
scaler = StandardScaler()
train_data['CumulativeHeatLoad'] = scaler.fit_transform(train_data[['CumulativeHeatLoad']])
val_data['CumulativeHeatLoad'] = scaler.transform(val_data[['CumulativeHeatLoad']])

# Define the single feature
features = ['CumulativeHeatLoad']

# Calculate the correlation between CumulativeHeatLoad and NormalizedDailyYield
cumulative_heatload_correlation = train_data['CumulativeHeatLoad'].corr(train_data[target])
print("Correlation between CumulativeHeatLoad and NormalizedDailyYield:", cumulative_heatload_correlation)

# Function to fit and get the posterior mean for CumulativeHeatLoad
def fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation):
    print(f"\nSelected features: {features}")

    # Calculate prior means for the features
    prior_mean_values = [cumulative_heatload_correlation]  # Use correlation as prior mean for CumulativeHeatLoad
    prior_std_values = [train_data[feature].std() for feature in features]
    
    # Define priors
    prior_mean = np.array([normalized_mean] + prior_mean_values).reshape(-1, 1)
    prior_std = np.array([np.sqrt(normalized_variance)] + prior_std_values).reshape(-1, 1)
    prior_cov = np.eye(len(features) + 1) * 0.01  # Smaller value for stronger prior confidence
    beta = 1 / normalized_variance

    # Print prior means and standard deviations
    print(f"\nPrior Mean and Standard Deviation for each feature:\n")
    for feature, mean, std in zip(['Off-set'] + features, prior_mean.flatten(), prior_std.flatten()):
        print(f"{feature}: mean = {mean}, std_dev = {std}")

    # Initialize and fit the model
    model = BayesianLinearRegression(
        dataframe=train_data,
        subject_name=farm_id,
        selected_features=features,
        target=target,
        subject_type=subject_type,
        prior_mean=prior_mean,
        prior_cov=prior_cov,
        beta=beta
    )
    result = model.fit_model()

    # Print posterior mean and standard deviation for each feature
    print(f"\nPosterior Mean and Standard Deviation for each feature:\n")
    for feature, stats in result.items():
        print(f"{feature}: mean = {stats['mu']}, std_dev = {stats['sigma']}")

    # Extract the posterior mean for CumulativeHeatLoad
    cumulative_heatload_posterior_mean = result['CumulativeHeatLoad']['mu']

    return cumulative_heatload_posterior_mean

# Train the model with the CumulativeHeatLoad feature on the training set and print results
cumulative_heatload_posterior_mean = fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmCumulativeHeatLoadMilkProduction': cumulative_heatload_posterior_mean
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 1.0022932145662267
Standard Deviation of NormalizedDailyYield: 0.12802764401772412
Variance of NormalizedDailyYield: 0.016391077632729092
Correlation between CumulativeHeatLoad and NormalizedDailyYield: -0.020363167770994367

Selected features: ['CumulativeHeatLoad']

Prior Mean and Standard Deviation for each feature:

Off-set: mean = 1.0022932145662267, std_dev = 0.12802764401772412
CumulativeHeatLoad: mean = -0.020363167770994367, std_dev = 1.0000038230026242

Posterior Mean and Standard Deviation for each feature:

Off-set: mean = 1.002442519991088, std_dev = 0.0003568408924133986
CumulativeHeatLoad: mean = -0.0026278684995927125, std_dev = 0.0003568408924133977


,FarmName_Pseudo,FarmCumulativeHeatLoadMilkProduction
0,5c06d92d,-0.002628
